In [1]:
# install required system dependencies
# install required system dependencies
!apt-get install -y xvfb x11-utils 
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyVirtualDisplay==2.0.* \
  PyOpenGL==3.1.* \
  PyOpenGL-accelerate==3.1.* \
  gym[box2d]==0.17.* 
!pip install pyglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 993 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 993 kB in 1s (1,329 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:

In [2]:
import gym
import numpy as np
import base64
import io
import IPython

In [44]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    import gym
    import numpy as np

    env = gym.make('CartPole-v0')

    max_number_of_steps = 200   
    
    goal_average_steps = 195
    num_consecutive_iterations = 100
    
    num_episodes = 300 
    last_time_steps = np.zeros(num_consecutive_iterations)  

    q_table = np.random.uniform(low=-1, high=1, size=(4 ** 4, env.action_space.n))

    def bins(clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]

    def digitize_state(observation):
        cart_pos, cart_v, pole_angle, pole_v = observation
        digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                     np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                     np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                     np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
        return sum([x * (4 ** i) for i, x in enumerate(digitized)])

    def get_action(state, action, observation, reward):
        
        next_state = digitize_state(observation)    
        next_action = np.argmax(q_table[next_state])    
        alpha = 0.12    
        gamma = 0.999   
        q_table[state, action] = (1 - alpha) * q_table[state, action] + alpha * (reward + gamma * max(q_table[next_state, :]))
        
        return next_action, next_state

    for episode in range(num_episodes):
        observation = env.reset()   
        state = digitize_state(observation)     
        action = np.argmax(q_table[state])      
        episode_reward = 0
        
        for t in range(max_number_of_steps):
            env.render()    
            observation, reward, done, info = env.step(action)  
            if done:
              reward = -200
            action, state = get_action(state, action, observation, reward)  
            episode_reward += reward
            if done:
                
                last_time_steps = np.hstack((last_time_steps[1:], [t + 1]))  
                if episode % 10 == 0:
                  print('%d Episode finished after %f time steps / mean %f' % (episode, t + 1, last_time_steps.mean()))
                break
                
            if (last_time_steps.mean() >= goal_average_steps):
                print('Episode %d train agent successfuly!' % episode)
                break

    print('Failed!')

0 Episode finished after 17.000000 time steps / mean 0.170000
10 Episode finished after 10.000000 time steps / mean 2.550000
20 Episode finished after 10.000000 time steps / mean 5.950000
30 Episode finished after 30.000000 time steps / mean 16.060000
40 Episode finished after 200.000000 time steps / mean 30.460000
50 Episode finished after 200.000000 time steps / mean 46.010000
60 Episode finished after 200.000000 time steps / mean 64.800000
70 Episode finished after 200.000000 time steps / mean 84.530000
80 Episode finished after 200.000000 time steps / mean 103.820000
90 Episode finished after 200.000000 time steps / mean 122.580000
100 Episode finished after 200.000000 time steps / mean 141.510000
110 Episode finished after 194.000000 time steps / mean 157.760000
120 Episode finished after 200.000000 time steps / mean 174.040000
130 Episode finished after 173.000000 time steps / mean 181.760000
140 Episode finished after 200.000000 time steps / mean 186.220000
150 Episode finished 